<a href="https://colab.research.google.com/github/stumbi/mir_nlp/blob/main/exercise_05_classificator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

  <div>
    <h1 align="center">Excercise 05 - Medical Information Retrieval 2023</h1>
  </div>
  <br />

Today, we are moving on, towards a machine learning approach for text classification. 

## Text classification <a class="anchor" id="first"></a>

In the following 3 weeks we are focussing on machine learning approaches on our classification task. Feel free to use any tool which helps you, as long as you can explain, what exactelly is happening, and why it is useful. Given, that you know the preprocessing steps from the past weeks and are able to apply them, we want you to use them now in order to develop a machine learning model for our classification problem.

### Requirements
* The notebook should run **without any error**, given that all packages are installed and the dataset is loaded. When we test it, we will adapt path definitions and might will install nessesary packages)
* Your training/validation script should only use the train split we give you.

### Evaluation
* For evalutation, you can use the function "test_model_performance" in this notebook for accuracy, precision, recall and F1-score. If you choose to use such evaluation, the predicted labels have to be hot-encoded: The output of your model should be a vector of probabilities for each class. 
### Your tasks

* Make an exploratory data analysis
* Develop a preprocessing pipeline
* Train and test one or several machine learning models
* Evaluate the algorithms with a metric of your choice 
* Visualize the outcome

* Prepare a presentation (or present this notebook) of around 10 minutes for our last session (6th of June)


You can start from here. To have a comparable evaluation between each group, we give you a fixed train and test split.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

!pip install datasets
!pip install transformers
!pip install --upgrade accelerate
!pip install evaluate
### loading the dataset ###

Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 18.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1

In [2]:
import pandas as pd
import numpy as np
import re
import torch
from torch import nn
import nltk
from nltk.corpus import stopwords
import plotly.express as px
import plotly.graph_objects as go
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split

import evaluate
from evaluate import evaluator

from collections import Counter

from datasets import Dataset, ClassLabel
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, pipeline, DataCollatorWithPadding, AutoTokenizer

nltk.download('stopwords')

df = pd.read_csv('/content/drive/MyDrive/Uni/3. Semester/MIR/DATA/mtsamples_clean.csv')

### creating train and test split ###


_X = df['transcription']
_y = df['medical_specialty']
_y_one_hot = pd.get_dummies(_y)

X, X_test, y_one_hot, _ = train_test_split(_X, _y_one_hot, test_size=0.2, random_state=123)
_, _, y_classes, _ = train_test_split(_X, _y, test_size=0.2, random_state=123)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Only use X and y_one_hot or y_classes for training purposes in the rest of the notebook. After running the whole notebook, there should be a prediction from your model, which took X_test as input to create the predictions. Each prediction has to be a vector of length 40.

In [3]:
def test_model_performance(y_pred):
    _, _, _, y_test = train_test_split(_X, _y_one_hot, test_size=0.2, random_state=123)

    # set highest to 1 and rest to 0
    #y_pred = np.argmax(y_pred, axis=1)

    print('Accuracy: ', accuracy_score(y_test, y_pred))
    print('Precision: ', precision_score(y_test, y_pred, average='weighted'))
    print('Recall: ', recall_score(y_test, y_pred, average='weighted'))
    print('F1: ', f1_score(y_test, y_pred, average='weighted'))

In [4]:
### performance of a random guesser ###

y_pred_dummy = np.zeros((len(X_test), 40))
# random predictions with sum 1
y_pred_dummy = y_pred_dummy + np.random.rand(y_pred_dummy.shape[0], y_pred_dummy.shape[1])
y_pred_dummy = y_pred_dummy / y_pred_dummy.sum(axis=1).reshape(-1, 1)

# set only highest value to 1 and rest to 0
y_pred_dummy = np.argmax(y_pred_dummy, axis=1)
y_pred_dummy = pd.get_dummies(y_pred_dummy).values

test_model_performance(y_pred_dummy)

Accuracy:  0.018
Precision:  0.047234548451611263
Recall:  0.018
F1:  0.022282959716363366


# Data Exploration

In [5]:
### your code ###
df.head()

,medical_specialty,transcription
0,Allergy / Immunology,"SUBJECTIVE:, This 23-year-old white female pr..."
1,Bariatrics,"PAST MEDICAL HISTORY:, He has difficulty climb..."
2,Bariatrics,"HISTORY OF PRESENT ILLNESS: , I have seen ABC ..."
3,Cardiovascular / Pulmonary,"2-D M-MODE: , ,1. Left atrial enlargement wit..."
4,Cardiovascular / Pulmonary,1. The left ventricular cavity size and wall ...


In [6]:
fig = px.histogram(df, x="medical_specialty")
fig.show()

In [7]:
eng_stopwords = stopwords.words('english')

nlp = English()
tokenizer = Tokenizer(nlp.vocab)

In [8]:
df['cleaned'] = df['transcription'].apply(lambda row: re.sub(r"[\W\d\s]", ' ', str(row)).lower())
df['cleaned'] = df['cleaned'].apply(lambda x: x.split())
df['stopwords_removed'] = df['cleaned'].apply(lambda x: [word for word in x if word not in eng_stopwords])
df['tokenized'] = df['stopwords_removed'].apply(lambda x: tokenizer(str(x)))


In [9]:
len_of_txts = df.tokenized.map(len)

fig = go.Figure(data=[go.Histogram(x=len_of_txts)])

fig.show()

In [10]:
df['word_count'] = len_of_txts
sub_df = df[['medical_specialty', 'word_count']]

sub_df.groupby(by=['medical_specialty']).sum()

,word_count
medical_specialty,
Allergy / Immunology,1897
Autopsy,5697
Bariatrics,4210
Cardiovascular / Pulmonary,98801
Chiropractic,7478
Consult - History and Phy.,172938
Cosmetic / Plastic Surgery,8433
Dentistry,7849
Dermatology,6948


In [11]:
full_text = []

for i in df['stopwords_removed']:
    full_text += i



Counter(full_text).most_common()[0:25]

[('patient', 24208),
 ('right', 11587),
 ('left', 11258),
 ('history', 9509),
 ('normal', 7526),
 ('procedure', 7463),
 ('placed', 7028),
 ('well', 6611),
 ('pain', 5976),
 ('mg', 4375),
 ('x', 4357),
 ('noted', 4348),
 ('also', 4337),
 ('time', 4287),
 ('c', 4132),
 ('using', 4123),
 ('blood', 3956),
 ('performed', 3953),
 ('skin', 3798),
 ('without', 3732),
 ('anesthesia', 3707),
 ('incision', 3601),
 ('used', 3554),
 ('removed', 3532),
 ('year', 3506)]

### Calculating the class weights with root inverse frequency

In [12]:
P = (df['medical_specialty'].value_counts() / len(df))
class_weights = np.sqrt(P.pow(-1))

Classifier

In [13]:
print('Number of individual classes:', len(set(df['medical_specialty'])))

Number of individual classes: 40


### Data Preparation for Transformer Learning

- creating label to id mapping
- creating train, validation and test dataset
- loading model and tokenizer
- creating custom tokenizer pipeline

In [14]:
label2id = {item : idx for idx, item in enumerate(set(df['medical_specialty']))}
id2label = {idx : item for idx, item in enumerate(set(df['medical_specialty']))}

In [15]:
X = df['stopwords_removed'].apply(lambda x: ' '.join(x))
y = df['medical_specialty']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.5, random_state=42)

X_val, X_test, y_val, y_test = train_test_split(
    X_test, y_test, test_size=0.5, random_state=42)

In [16]:
bert_df_train = pd.DataFrame()
bert_df_val = pd.DataFrame()
bert_df_test = pd.DataFrame()
bert_df_train['sentence'] = X_train
bert_df_train['label'] = y_train
bert_df_val['sentence'] = X_val
bert_df_val['label'] = y_val
bert_df_test['sentence'] = X_test
bert_df_test['label'] = y_test

In [17]:
train_dataset = Dataset.from_pandas(bert_df_train)
val_dataset = Dataset.from_pandas(bert_df_val)
test_dataset = Dataset.from_pandas(bert_df_test)

In [18]:
# load the tokenizer

tokenizer = AutoTokenizer.from_pretrained("dmis-lab/biobert-v1.1")

tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.pad_token = ['0']
labels = ClassLabel(names=list(set(y_train.values)))

# define the tokenization as a function

def preprocess_function(examples):
    tokens = tokenizer(examples["sentence"], truncation=True, padding=True, max_length=512)
    
    tokens['label'] = labels.str2int(examples['label'])
    return tokens

tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_val = val_dataset.map(preprocess_function, batched=True)
tokenized_test = test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/2499 [00:00<?, ? examples/s]

Map:   0%|          | 0/1250 [00:00<?, ? examples/s]

Map:   0%|          | 0/1250 [00:00<?, ? examples/s]

In [19]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


model = AutoModelForSequenceClassification.from_pretrained("dmis-lab/biobert-v1.1", num_labels=40, id2label=id2label, label2id=label2id)
model = model.to('cuda')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dmis-lab/biobert-v1.1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


We decided to use the BioBERT model trained on PubMed text data. While LLMs could achieve a better performance, the BERT Models are small enough to be trained in a sufficient time. To improve at least a bit, we then decided to use a model which is already familiar with medical texts.

### Transformer Fine Tuning 

- creating custom loss function, due to unbalanced training dataset
- define compute metrics function
- setting training arguments
- setting trainer arguments

In [20]:
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get('logits').to(torch.float)
      
        # compute custom loss TODO change to_listmor to_tensor or without list()
        loss_fct = nn.CrossEntropyLoss(weight=torch.tensor(list(class_weights.loc[label2id.keys()].values)).to(torch.float).to('cuda'))
        loss = loss_fct(logits.view(-1, 40).to(torch.float), labels.view(-1))
        
        return (loss, outputs) if return_outputs else loss

def compute_metrics(eval_preds):
    metric = evaluate.load("accuracy", "f1")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [21]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    gradient_accumulation_steps=16,
    dataloader_num_workers=2,
    gradient_checkpointing=True,
    evaluation_strategy="epoch"
)

trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)


In [22]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning:

This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
0,No log,3.422020,0.234400
1,No log,3.228836,0.246400
2,No log,3.074163,0.275200
3,No log,2.947326,0.317600
4,No log,2.819585,0.348000
5,No log,2.704924,0.362400
6,No log,2.636353,0.366400
7,No log,2.566671,0.376800
8,No log,2.542567,0.380800
9,No log,2.541905,0.380800


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `_

TrainOutput(global_step=90, training_loss=2.895725843641493, metrics={'train_runtime': 3160.2059, 'train_samples_per_second': 7.908, 'train_steps_per_second': 0.028, 'total_flos': 6033352519360512.0, 'train_loss': 2.895725843641493, 'epoch': 9.17})

### Evaluate Training

- printing accuracy
- calculating f1 score

In [23]:
task_evaluator = evaluator("text-classification")

metric = evaluate.load("accuracy")
eval_results = task_evaluator.compute(
    model_or_pipeline=model,
    data=tokenized_test,
    input_column='sentence',
    label_column='label',
    tokenizer=tokenizer,
    label_mapping=label2id,
    metric=metric
)

print(eval_results)

Xformers is not installed correctly. If you want to use memorry_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


{'accuracy': 0.3528, 'total_time_in_seconds': 33.215002852, 'samples_per_second': 37.63359604603294, 'latency_in_seconds': 0.026572002281599998}


In [ ]:
from datasets import load_metric



classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)
tokenizer_kwargs = {'max_length':512}

model_predictions = classifier(tokenized_test['sentence'], **tokenizer_kwargs)


In [ ]:
model_predictions_temp = [i['label'] for i in model_predictions]

for idx,j  in enumerate(model_predictions_temp):
    model_predictions_temp[idx] = label2id[j]
model_predictions_temp

In [51]:
metric = load_metric('f1')

final_score = metric.compute(predictions=model_predictions_temp, references=tokenized_test['label'], average="weighted")
final_score

{'f1': 0.3025185551488107}